# Prototype of reaction-diffusion on temporal networks

Install the packages

In [ ]:
Pkg.update()
Pkg.add("Distributions")
Pkg.add("LightGraphs")
Pkg.add("SimJulia")
Pkg.checkout("SimJulia")

Making packages available in notebook

In [ ]:
using Distributions
using LightGraphs
using ResumableFunctions
using SimJulia

Define a type to hold the state of the simulation

In [ ]:
mutable struct SimulationState
    network :: DiGraph
    link_up :: Event
end

Define the temporal behaviour of the network

In [ ]:
@resumable function edge_behaviour(sim::Simulation, initial_state::Bool, src::Int, dst::Int, sim_state::SimulationState)
    up_distribution = Exponential(1.0) # other distributions are available
    down_distribution = Exponential(1.0) # other distributions are available
    next_state = initial_state
    while true
        if next_state
            println(now(sim), ": Edge from ", src, " to ", dst, " up")
            add_edge!(sim_state.network, src, dst)
            next_state = false
            succeed(sim_state.link_up, value=src)
            sim_state.link_up = Event(sim)
            @yield Timeout(sim, rand(up_distribution))
        else
            println(now(sim), ": Edge from ", src, " to ", dst, " down")
            rem_edge!(sim_state.network, src, dst)
            next_state = true
            @yield Timeout(sim, rand(down_distribution))
        end
    end
end

Define the walker behaviour

In [ ]:
@resumable function walker_behaviour(sim::Simulation, walker::Int, vertex::Int, sim_state::SimulationState)
    wait_distribution = Exponential(1.0) # other distributions are available
    while true
        println(now(sim), ": Walker ", walker, " arrived in vertex ", vertex)
        @yield Timeout(sim, rand(wait_distribution))
        connections = out_neighbors(sim_state.network, vertex)
        println(now(sim), ": Walker ", walker, " in vertex ", vertex, " has connections to ", connections)
        #isempty(connections) && continue # no connections available, so we wait another period
        if isempty(connections)
            src = 0
            while src != vertex
                println(now(sim), ": Walker ", walker, " in vertex ", vertex, " is waiting for connections")
                src = @yield sim_state.link_up
            end
        end
        connection_distribution = DiscreteUniform(1, length(connections))
        vertex = connections[Int(rand(connection_distribution))]
    end
end

Define the initial simulation settings

In [ ]:
function play(run_time::Float64, nr_edges::Int, nr_walkers::Int)
    sim = Simulation()
    sim_state = SimulationState(DiGraph(nr_edges), Event(sim))
    edge_distribution = Bernoulli(0.5) # modify parameter to change the density of the initial network
    walker_distribution = DiscreteUniform(1, nr_edges)
    for src in 1:nr_edges
        for dst in 1:nr_edges
            src == dst && continue # self loops are not allowed
            state = Bool(rand(edge_distribution))
            @coroutine edge_behaviour(sim, state, src, dst, sim_state)
        end
    end
    for walker in 1:nr_walkers
        vertex = Int(rand(walker_distribution))
        @coroutine walker_behaviour(sim, walker, vertex, sim_state)
    end
    run(sim, run_time)
end

In [ ]:
play(5.0, 3, 2)